In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Concatenate, Conv2D, Lambda, Layer, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.models import Model
import numpy as np 
import pandas as pd
import pickle
import os
import cv2

In [17]:
with open('training_pairs.pkl', 'rb') as f:
    train_pairs = pickle.load(f)

with open('testing_pairs.pkl', 'rb') as f:
    test_pairs = pickle.load(f)

train_pairs_a = train_pairs['train_a']
train_pairs_b = train_pairs['train_b']
train_labels = train_pairs['train_labels']

test_pairs_a = test_pairs['test_a']
test_pairs_b = test_pairs['test_b']
test_labels = test_pairs['test_labels']

In [18]:
def parallel_vectors():
    
    imgs = Input(shape=(105, 105, 3))
        
    cv1 = Conv2D(64, (10,10), activation="relu")(imgs)
    mp1 = MaxPooling2D(2,2)(cv1)

    cv2 = Conv2D(128, (7,7), activation="relu")(mp1)
    mp2 = MaxPooling2D(2,2)(cv2)

    cv3 = Conv2D(128, (4,4), activation="relu")(mp2)
    mp3 = MaxPooling2D(2,2)(cv3)

    cv4 = Conv2D(256, (4,4), activation="relu")(mp3)
    flat = Flatten()(cv4)
    dense = Dense(4096, activation="sigmoid")(flat)
    
    return Model(inputs=[imgs], outputs=[dense], name="parallel_vectors")

In [19]:
embedding = parallel_vectors()
embedding.summary()

Model: "parallel_vectors"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 105, 105, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 96, 96, 64)     │        19,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 42, 42, 128)    │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 21, 21, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 18, 18, 128)    │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 6, 6, 256)      │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    37,752,832 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
def distance(vectors):
    embedded_example1, embedded_example2 = vectors
    embedded_example1 = tf.squeeze(tf.convert_to_tensor(embedded_example1))
    embedded_example2 = tf.squeeze(tf.convert_to_tensor(embedded_example2))
    return tf.math.abs(embedded_example1 - embedded_example2)

In [31]:
def siamese_model(): 

    input_image = Input(name='input_img', shape=(105,105,3))
    anchor_image = Input(name='validation_img', shape=(105,105,3))
    
    embed = parallel_vectors()
    embedded_input = embed(input_image)
    embedded_anchor = embed(anchor_image)

    embedded_input = tf.convert_to_tensor(embedded_input)
    embedded_anchor = tf.convert_to_tensor(embedded_anchor)
    
    siamese_layer = Concatenate()([embedded_input, embedded_anchor])
    dense = Dense(4096, activation='relu')(siamese_layer)
    classifier = Dense(1, activation='sigmoid')(dense)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [32]:
model = siamese_model()

ValueError: TypeError: object of type 'KerasTensor' has no len()


In [27]:
model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_img           │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_img      │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ parallel_vectors    │ (None, 4096)      │ 38,960,448 │ input_img[0][0],  │
│ (Functional)        │                   │            │ validation_img[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ distance_layer      │ (None, 32, 4096)  │          0 │ parallel_vectors… │
│ (Lambda)            │                   │            │ parallel_vectors… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32, 4096)  │ 16,781,312 │ distance_layer[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32, 1)     │      4,097 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 55,745,857 (212.65 MB)

 Trainable params: 55,745,857 (212.65 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit([train_pairs_a, train_pairs_b], train_labels, epochs=10, batch_size=50, validation_data=([test_pairs_a, test_pairs_b], test_labels))

Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 270s 3s/step - accuracy: 0.5119 - loss: 0.6964 - val_accuracy: 0.5158 - val_loss: 0.6930
Epoch 2/10
45/96 ━━━━━━━━━━━━━━━━━━━━ 2:18 3s/step - accuracy: 0.5089 - loss: 0.6931

KeyboardInterrupt: 

In [123]:
print("Shape of train_pairs_a:", train_pairs_a.shape)
print("Shape of train_pairs_b:", train_pairs_b.shape)
print("Shape of test_pairs_a:", test_pairs_a.shape)
print("Shape of test_pairs_b:", test_pairs_b.shape)
print("Shape of train_labels:", train_labels.shape)
print("Shape of test_labels:", test_labels.shape)

Shape of train_pairs_a: (4800, 105, 105, 3)
Shape of train_pairs_b: (4800, 105, 105, 3)
Shape of test_pairs_a: (1200, 105, 105, 3)
Shape of test_pairs_b: (1200, 105, 105, 3)
Shape of train_labels: (4800,)
Shape of test_labels: (1200,)
